In [1]:
import pandas as pd
from rdkit.Chem import AllChem as Chem
import pandas_utils as pu
from tqdm import tqdm
from functools import lru_cache
import numpy as np
from rdkit import DataStructs
from myopic_mces.myopic_mces import MCES

In [2]:
original_file = r"C:\git\msn_library\data\library_matching\20231031\lib_20231031_nih_library_pos_all_lib_MS2.mgf__19241_spectra__single_scans.csv"
match_file = r"C:\git\msn_library\data\library_matching\20231031\match_top10_20231031_nih_library_pos_all_lib_MS2.mgf__19241_spectra__single_scans_cleaned.csv"
outfile = r"C:\git\msn_library\data\library_matching\20231031\match_top10_20231031_nih_library_pos_all_lib_MS2.mgf__19241_spectra__single_scans_comparison.csv"

In [3]:
original_df = pu.read_dataframe(original_file).rename(columns={"inchi_key": "inchikey"})

In [4]:
original_df

,id,compound_name,adduct,score,precursor_mz,ion_mobility,ccs,rt,mol_formula,smiles,inchi,inchikey,method
0,14441,CHEBI:181155,[M+H]+,1.0,235.16926,NaN,NaN,0.1655,C15H22O2,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,InChI=1S/C15H22O2/c1-9-4-5-12-11(13(16)17)6-10...,DCFDRCCHOOORSB-UHFFFAOYSA-N,spectral_library_annotation
1,14442,CHEBI:181155,[M+H]+,1.0,235.16926,NaN,NaN,0.1655,C15H22O2,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,InChI=1S/C15H22O2/c1-9-4-5-12-11(13(16)17)6-10...,DCFDRCCHOOORSB-UHFFFAOYSA-N,spectral_library_annotation
2,14443,CHEBI:181155,[M+H]+,1.0,235.16926,NaN,NaN,0.1655,C15H22O2,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,InChI=1S/C15H22O2/c1-9-4-5-12-11(13(16)17)6-10...,DCFDRCCHOOORSB-UHFFFAOYSA-N,spectral_library_annotation
3,14444,CHEBI:181155,[M+H]+,1.0,235.16926,NaN,NaN,0.1655,C15H22O2,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,InChI=1S/C15H22O2/c1-9-4-5-12-11(13(16)17)6-10...,DCFDRCCHOOORSB-UHFFFAOYSA-N,spectral_library_annotation
4,14445,CHEBI:181155,[M+H]+,1.0,235.16926,NaN,NaN,0.1655,C15H22O2,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,InChI=1S/C15H22O2/c1-9-4-5-12-11(13(16)17)6-10...,DCFDRCCHOOORSB-UHFFFAOYSA-N,spectral_library_annotation
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19236,14286,ISOPETASOL,[M+H]+,1.0,235.16926,NaN,NaN,2.4680,C15H22O2,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,InChI=1S/C15H22O2/c1-9(2)12-8-15(4)10(3)13(16)...,TVDMUSYVWJLIDK-PSOPSSQASA-N,spectral_library_annotation
19237,14287,ISOPETASOL,[M+H]+,1.0,235.16926,NaN,NaN,2.4680,C15H22O2,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,InChI=1S/C15H22O2/c1-9(2)12-8-15(4)10(3)13(16)...,TVDMUSYVWJLIDK-PSOPSSQASA-N,spectral_library_annotation
19238,14288,ISOPETASOL,[M+H]+,1.0,235.16926,NaN,NaN,2.4680,C15H22O2,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,InChI=1S/C15H22O2/c1-9(2)12-8-15(4)10(3)13(16)...,TVDMUSYVWJLIDK-PSOPSSQASA-N,spectral_library_annotation
19239,14289,ISOPETASOL,[M+H]+,1.0,235.16926,NaN,NaN,2.4680,C15H22O2,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,InChI=1S/C15H22O2/c1-9(2)12-8-15(4)10(3)13(16)...,TVDMUSYVWJLIDK-PSOPSSQASA-N,spectral_library_annotation


In [5]:
match_df = pu.read_dataframe(match_file)
match_df = match_df.dropna(subset="inchikey")
match_df

,id,compound_name,adduct,score,precursor_mz,ion_mobility,ccs,rt,mol_formula,smiles,...,inchikey,method,canonical_smiles,isomeric_smiles,smarts,monoisotopic_mass,split_inchikey,formula,logp,match
0,14441,"NCGC00381379-01_C15H22O2_(4aR,5R,6R)-6-Hydroxy...",[M+H]+,0.823,235.16900,NaN,NaN,NaN,NaN,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,...,TVDMUSYVWJLIDK-PSOPSSQASA-N,spectral_library_annotation,CC(C)=C1CC2(C)C(=CC1=O)CCC(O)C2C,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,[#6]-[#6](-[#6])=[#6]1-[#6]-[#6@@]2(-[#6])-[#6...,234.161980,TVDMUSYVWJLIDK,C15H22O2,3.0191,True
1,14441,"NCGC00381379-01_C15H22O2_(4aR,5R,6R)-6-Hydroxy...",[M+H]+,0.823,235.16900,NaN,NaN,NaN,C15H22O2,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,...,TVDMUSYVWJLIDK-PSOPSSQASA-N,spectral_library_annotation,CC(C)=C1CC2(C)C(=CC1=O)CCC(O)C2C,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,[#6]-[#6](-[#6])=[#6]1-[#6]-[#6@@]2(-[#6])-[#6...,234.161980,TVDMUSYVWJLIDK,C15H22O2,3.0191,True
2,14441,"NCGC00385861-01_C15H24O3_1,2,4-Metheno-1H-inde...",[M-H2O+H]+,0.726,235.16900,NaN,NaN,NaN,NaN,CC(C)(O)C1CCC2(C)C3C(O)C4C(C13)C42CO,...,ORUMCUXABQCFEG-UHFFFAOYSA-N,spectral_library_annotation,CC(C)(O)C1CCC2(C)C3C(O)C4C(C13)C42CO,CC(C)(O)C1CCC2(C)C3C(O)C4C(C13)C42CO,[#6]-[#6](-[#6])(-[#8])-[#6]1-[#6]-[#6]-[#6]2(...,252.172545,ORUMCUXABQCFEG,C15H24O3,1.0188,True
3,14441,"NCGC00385861-01_C15H24O3_1,2,4-Metheno-1H-inde...",[M-H2O+H]+,0.726,235.16900,NaN,NaN,NaN,C15H24O3,CC(C)(O)C1CCC2(C)C3C(O)C4C(C13)C42CO,...,ORUMCUXABQCFEG-UHFFFAOYSA-N,spectral_library_annotation,CC(C)(O)C1CCC2(C)C3C(O)C4C(C13)C42CO,CC(C)(O)C1CCC2(C)C3C(O)C4C(C13)C42CO,[#6]-[#6](-[#6])(-[#8])-[#6]1-[#6]-[#6]-[#6]2(...,252.172545,ORUMCUXABQCFEG,C15H24O3,1.0188,True
4,14441,NCGC00385860-01_C15H22O2_3-Hydroxy-5-isopropyl...,[M+H]+,0.722,235.16900,NaN,NaN,NaN,NaN,CC1=CC(=O)C(=C(C)C)CC2C1CCC2(C)O,...,RHBOHEXDGUVIIY-UHFFFAOYSA-N,spectral_library_annotation,CC1=CC(=O)C(=C(C)C)CC2C1CCC2(C)O,CC1=CC(=O)C(=C(C)C)CC2C1CCC2(C)O,[#6]-[#6]1=[#6]-[#6](=[#8])-[#6](=[#6](-[#6])-...,234.161980,RHBOHEXDGUVIIY,C15H22O2,3.0191,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65940,14290,"NCGC00381379-01_C15H22O2_(4aR,5R,6R)-6-Hydroxy...",[M+H]+,0.674,235.16900,NaN,NaN,NaN,NaN,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,...,TVDMUSYVWJLIDK-PSOPSSQASA-N,spectral_library_annotation,CC(C)=C1CC2(C)C(=CC1=O)CCC(O)C2C,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,[#6]-[#6](-[#6])=[#6]1-[#6]-[#6@@]2(-[#6])-[#6...,234.161980,TVDMUSYVWJLIDK,C15H22O2,3.0191,True
65941,14290,Curcumenol,[M+H]+,0.646,235.16900,NaN,NaN,NaN,NaN,CC1=CC2(O)OC3(CC2=C(C)C)C(C)CCC13,...,ISFMXVMWEWLJGJ-UHFFFAOYSA-N,spectral_library_annotation,CC1=CC2(O)OC3(CC2=C(C)C)C(C)CCC13,CC1=CC2(O)OC3(CC2=C(C)C)C(C)CCC13,[#6]-[#6]1=[#6]-[#6]2(-[#8])-[#8]-[#6]3(-[#6]-...,234.161980,ISFMXVMWEWLJGJ,C15H22O2,3.1765,True
65942,14290,Curcumenol,[M+H]+,0.646,235.16925,NaN,NaN,NaN,C15H22O2,CC1=CC2(O)OC3(CC2=C(C)C)C(C)CCC13,...,ISFMXVMWEWLJGJ-UHFFFAOYSA-N,spectral_library_annotation,CC1=CC2(O)OC3(CC2=C(C)C)C(C)CCC13,CC1=CC2(O)OC3(CC2=C(C)C)C(C)CCC13,[#6]-[#6]1=[#6]-[#6]2(-[#8])-[#8]-[#6]3(-[#6]-...,234.161980,ISFMXVMWEWLJGJ,C15H22O2,3.1765,True
65943,14290,"Massbank:AC000285 Walleminone|(1R,4S,5S,6R,9R)...",[M-H2O+H]+,0.634,235.16900,NaN,NaN,NaN,NaN,C=C1C[C@@H](O)[C@@H](O)[C@H](C)C(=O)C[C@@H]2[C...,...,NGQXJSTYWWTPOG-VGYDOTAVSA-N,spectral_library_annotation,C=C1CC(O)C(O)C(C)C(=O)CC2C1CC2(C)C,C=C1C[C@@H](O)[C@@H](O)[C@H](C)C(=O)C[C@@H]2[C...,[#6]=[#6]1-[#6]-[#6@@H](-[#8])-[#6@@H](-[#8])-...,252.172545,NGQXJSTYWWTPOG,C15H24O3,1.9257,True


- Merge both frames on id (suffix for library match)
- equality for inchikeys
- keep only ids without equal hit
- cache for similarity (@lru cache)
- similarity for structure (tanimoto, mces)
- 

In [6]:
merged = pd.merge(original_df, match_df, on="id", how="outer", suffixes=("", "_match"))
merged

,id,compound_name,adduct,score,precursor_mz,ion_mobility,ccs,rt,mol_formula,smiles,...,inchikey_match,method_match,canonical_smiles,isomeric_smiles,smarts,monoisotopic_mass,split_inchikey,formula,logp,match
0,14441,CHEBI:181155,[M+H]+,1.0,235.16926,NaN,NaN,0.1655,C15H22O2,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,...,TVDMUSYVWJLIDK-PSOPSSQASA-N,spectral_library_annotation,CC(C)=C1CC2(C)C(=CC1=O)CCC(O)C2C,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,[#6]-[#6](-[#6])=[#6]1-[#6]-[#6@@]2(-[#6])-[#6...,234.161980,TVDMUSYVWJLIDK,C15H22O2,3.0191,True
1,14441,CHEBI:181155,[M+H]+,1.0,235.16926,NaN,NaN,0.1655,C15H22O2,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,...,TVDMUSYVWJLIDK-PSOPSSQASA-N,spectral_library_annotation,CC(C)=C1CC2(C)C(=CC1=O)CCC(O)C2C,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,[#6]-[#6](-[#6])=[#6]1-[#6]-[#6@@]2(-[#6])-[#6...,234.161980,TVDMUSYVWJLIDK,C15H22O2,3.0191,True
2,14441,CHEBI:181155,[M+H]+,1.0,235.16926,NaN,NaN,0.1655,C15H22O2,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,...,ORUMCUXABQCFEG-UHFFFAOYSA-N,spectral_library_annotation,CC(C)(O)C1CCC2(C)C3C(O)C4C(C13)C42CO,CC(C)(O)C1CCC2(C)C3C(O)C4C(C13)C42CO,[#6]-[#6](-[#6])(-[#8])-[#6]1-[#6]-[#6]-[#6]2(...,252.172545,ORUMCUXABQCFEG,C15H24O3,1.0188,True
3,14441,CHEBI:181155,[M+H]+,1.0,235.16926,NaN,NaN,0.1655,C15H22O2,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,...,ORUMCUXABQCFEG-UHFFFAOYSA-N,spectral_library_annotation,CC(C)(O)C1CCC2(C)C3C(O)C4C(C13)C42CO,CC(C)(O)C1CCC2(C)C3C(O)C4C(C13)C42CO,[#6]-[#6](-[#6])(-[#8])-[#6]1-[#6]-[#6]-[#6]2(...,252.172545,ORUMCUXABQCFEG,C15H24O3,1.0188,True
4,14441,CHEBI:181155,[M+H]+,1.0,235.16926,NaN,NaN,0.1655,C15H22O2,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,...,RHBOHEXDGUVIIY-UHFFFAOYSA-N,spectral_library_annotation,CC1=CC(=O)C(=C(C)C)CC2C1CCC2(C)O,CC1=CC(=O)C(=C(C)C)CC2C1CCC2(C)O,[#6]-[#6]1=[#6]-[#6](=[#8])-[#6](=[#6](-[#6])-...,234.161980,RHBOHEXDGUVIIY,C15H22O2,3.0191,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71990,14290,ISOPETASOL,[M+H]+,1.0,235.16926,NaN,NaN,2.4680,C15H22O2,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,...,TVDMUSYVWJLIDK-PSOPSSQASA-N,spectral_library_annotation,CC(C)=C1CC2(C)C(=CC1=O)CCC(O)C2C,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,[#6]-[#6](-[#6])=[#6]1-[#6]-[#6@@]2(-[#6])-[#6...,234.161980,TVDMUSYVWJLIDK,C15H22O2,3.0191,True
71991,14290,ISOPETASOL,[M+H]+,1.0,235.16926,NaN,NaN,2.4680,C15H22O2,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,...,ISFMXVMWEWLJGJ-UHFFFAOYSA-N,spectral_library_annotation,CC1=CC2(O)OC3(CC2=C(C)C)C(C)CCC13,CC1=CC2(O)OC3(CC2=C(C)C)C(C)CCC13,[#6]-[#6]1=[#6]-[#6]2(-[#8])-[#8]-[#6]3(-[#6]-...,234.161980,ISFMXVMWEWLJGJ,C15H22O2,3.1765,True
71992,14290,ISOPETASOL,[M+H]+,1.0,235.16926,NaN,NaN,2.4680,C15H22O2,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,...,ISFMXVMWEWLJGJ-UHFFFAOYSA-N,spectral_library_annotation,CC1=CC2(O)OC3(CC2=C(C)C)C(C)CCC13,CC1=CC2(O)OC3(CC2=C(C)C)C(C)CCC13,[#6]-[#6]1=[#6]-[#6]2(-[#8])-[#8]-[#6]3(-[#6]-...,234.161980,ISFMXVMWEWLJGJ,C15H22O2,3.1765,True
71993,14290,ISOPETASOL,[M+H]+,1.0,235.16926,NaN,NaN,2.4680,C15H22O2,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,...,NGQXJSTYWWTPOG-VGYDOTAVSA-N,spectral_library_annotation,C=C1CC(O)C(O)C(C)C(=O)CC2C1CC2(C)C,C=C1C[C@@H](O)[C@@H](O)[C@H](C)C(=O)C[C@@H]2[C...,[#6]=[#6]1-[#6]-[#6@@H](-[#8])-[#6@@H](-[#8])-...,252.172545,NGQXJSTYWWTPOG,C15H24O3,1.9257,True


In [121]:
merged["equal_inchikey"] = merged["inchikey"] == merged["inchikey_match"]
merged

,id,compound_name,adduct,score,precursor_mz,ion_mobility,ccs,rt,mol_formula,smiles,...,method_match,canonical_smiles,isomeric_smiles,smarts,monoisotopic_mass,split_inchikey,formula,logp,match,equal_inchikey
0,14441,CHEBI:181155,[M+H]+,1.0,235.16926,NaN,NaN,0.1655,C15H22O2,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,...,spectral_library_annotation,CC(C)=C1CC2(C)C(=CC1=O)CCC(O)C2C,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,[#6]-[#6](-[#6])=[#6]1-[#6]-[#6@@]2(-[#6])-[#6...,234.161980,TVDMUSYVWJLIDK,C15H22O2,3.0191,True,False
1,14441,CHEBI:181155,[M+H]+,1.0,235.16926,NaN,NaN,0.1655,C15H22O2,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,...,spectral_library_annotation,CC(C)=C1CC2(C)C(=CC1=O)CCC(O)C2C,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,[#6]-[#6](-[#6])=[#6]1-[#6]-[#6@@]2(-[#6])-[#6...,234.161980,TVDMUSYVWJLIDK,C15H22O2,3.0191,True,False
2,14441,CHEBI:181155,[M+H]+,1.0,235.16926,NaN,NaN,0.1655,C15H22O2,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,...,spectral_library_annotation,CC(C)(O)C1CCC2(C)C3C(O)C4C(C13)C42CO,CC(C)(O)C1CCC2(C)C3C(O)C4C(C13)C42CO,[#6]-[#6](-[#6])(-[#8])-[#6]1-[#6]-[#6]-[#6]2(...,252.172545,ORUMCUXABQCFEG,C15H24O3,1.0188,True,False
3,14441,CHEBI:181155,[M+H]+,1.0,235.16926,NaN,NaN,0.1655,C15H22O2,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,...,spectral_library_annotation,CC(C)(O)C1CCC2(C)C3C(O)C4C(C13)C42CO,CC(C)(O)C1CCC2(C)C3C(O)C4C(C13)C42CO,[#6]-[#6](-[#6])(-[#8])-[#6]1-[#6]-[#6]-[#6]2(...,252.172545,ORUMCUXABQCFEG,C15H24O3,1.0188,True,False
4,14441,CHEBI:181155,[M+H]+,1.0,235.16926,NaN,NaN,0.1655,C15H22O2,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,...,spectral_library_annotation,CC1=CC(=O)C(=C(C)C)CC2C1CCC2(C)O,CC1=CC(=O)C(=C(C)C)CC2C1CCC2(C)O,[#6]-[#6]1=[#6]-[#6](=[#8])-[#6](=[#6](-[#6])-...,234.161980,RHBOHEXDGUVIIY,C15H22O2,3.0191,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71990,14290,ISOPETASOL,[M+H]+,1.0,235.16926,NaN,NaN,2.4680,C15H22O2,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,...,spectral_library_annotation,CC(C)=C1CC2(C)C(=CC1=O)CCC(O)C2C,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,[#6]-[#6](-[#6])=[#6]1-[#6]-[#6@@]2(-[#6])-[#6...,234.161980,TVDMUSYVWJLIDK,C15H22O2,3.0191,True,True
71991,14290,ISOPETASOL,[M+H]+,1.0,235.16926,NaN,NaN,2.4680,C15H22O2,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,...,spectral_library_annotation,CC1=CC2(O)OC3(CC2=C(C)C)C(C)CCC13,CC1=CC2(O)OC3(CC2=C(C)C)C(C)CCC13,[#6]-[#6]1=[#6]-[#6]2(-[#8])-[#8]-[#6]3(-[#6]-...,234.161980,ISFMXVMWEWLJGJ,C15H22O2,3.1765,True,False
71992,14290,ISOPETASOL,[M+H]+,1.0,235.16926,NaN,NaN,2.4680,C15H22O2,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,...,spectral_library_annotation,CC1=CC2(O)OC3(CC2=C(C)C)C(C)CCC13,CC1=CC2(O)OC3(CC2=C(C)C)C(C)CCC13,[#6]-[#6]1=[#6]-[#6]2(-[#8])-[#8]-[#6]3(-[#6]-...,234.161980,ISFMXVMWEWLJGJ,C15H22O2,3.1765,True,False
71993,14290,ISOPETASOL,[M+H]+,1.0,235.16926,NaN,NaN,2.4680,C15H22O2,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,...,spectral_library_annotation,C=C1CC(O)C(O)C(C)C(=O)CC2C1CC2(C)C,C=C1C[C@@H](O)[C@@H](O)[C@H](C)C(=O)C[C@@H]2[C...,[#6]=[#6]1-[#6]-[#6@@H](-[#8])-[#6@@H](-[#8])-...,252.172545,NGQXJSTYWWTPOG,C15H24O3,1.9257,True,False


In [122]:
# filter dataframe: get rows with equal inchikey true 
# store id in set if true
# filter dataframe: remove all rows with ids in set

filtered = merged[merged["equal_inchikey"]]
id_with_identity_match = set(filtered["id"])

merged["has_identical_inchikey"] = [id in id_with_identity_match for id in merged["id"]]
filtered = merged[~merged["has_identical_inchikey"] & merged["match"]].copy()
filtered

,id,compound_name,adduct,score,precursor_mz,ion_mobility,ccs,rt,mol_formula,smiles,...,canonical_smiles,isomeric_smiles,smarts,monoisotopic_mass,split_inchikey,formula,logp,match,equal_inchikey,has_identical_inchikey
0,14441,CHEBI:181155,[M+H]+,1.0,235.16926,NaN,NaN,0.165500,C15H22O2,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,...,CC(C)=C1CC2(C)C(=CC1=O)CCC(O)C2C,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,[#6]-[#6](-[#6])=[#6]1-[#6]-[#6@@]2(-[#6])-[#6...,234.161980,TVDMUSYVWJLIDK,C15H22O2,3.01910,True,False,False
1,14441,CHEBI:181155,[M+H]+,1.0,235.16926,NaN,NaN,0.165500,C15H22O2,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,...,CC(C)=C1CC2(C)C(=CC1=O)CCC(O)C2C,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,[#6]-[#6](-[#6])=[#6]1-[#6]-[#6@@]2(-[#6])-[#6...,234.161980,TVDMUSYVWJLIDK,C15H22O2,3.01910,True,False,False
2,14441,CHEBI:181155,[M+H]+,1.0,235.16926,NaN,NaN,0.165500,C15H22O2,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,...,CC(C)(O)C1CCC2(C)C3C(O)C4C(C13)C42CO,CC(C)(O)C1CCC2(C)C3C(O)C4C(C13)C42CO,[#6]-[#6](-[#6])(-[#8])-[#6]1-[#6]-[#6]-[#6]2(...,252.172545,ORUMCUXABQCFEG,C15H24O3,1.01880,True,False,False
3,14441,CHEBI:181155,[M+H]+,1.0,235.16926,NaN,NaN,0.165500,C15H22O2,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,...,CC(C)(O)C1CCC2(C)C3C(O)C4C(C13)C42CO,CC(C)(O)C1CCC2(C)C3C(O)C4C(C13)C42CO,[#6]-[#6](-[#6])(-[#8])-[#6]1-[#6]-[#6]-[#6]2(...,252.172545,ORUMCUXABQCFEG,C15H24O3,1.01880,True,False,False
4,14441,CHEBI:181155,[M+H]+,1.0,235.16926,NaN,NaN,0.165500,C15H22O2,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,...,CC1=CC(=O)C(=C(C)C)CC2C1CCC2(C)O,CC1=CC(=O)C(=C(C)C)CC2C1CCC2(C)O,[#6]-[#6]1=[#6]-[#6](=[#8])-[#6](=[#6](-[#6])-...,234.161980,RHBOHEXDGUVIIY,C15H22O2,3.01910,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71702,9735,Lithospermic acid,[M+NH4]+,1.0,556.14495,NaN,NaN,1.888167,C27H22O12,O=C(/C=C/c1c2c(c(O)cc1)O[C@H](c1cc(O)c(O)cc1)[...,...,O=C(C=Cc1c2c(c(O)cc1)OC(c1ccc(O)c(O)c1)C2C(=O)...,O=C(/C=C/c1c2c(c(O)cc1)OC(c1ccc(O)c(O)c1)C2C(=...,[#8]=[#6](/[#6]=[#6]/[#6]1:[#6]2:[#6](:[#6](-[...,538.111126,UJZQBMQZMKFSRV,C27H22O12,-0.22450,True,False,False
71743,3451,alternarian acid,[M+H]+,1.0,321.06049,NaN,NaN,1.902833,C15H12O8,COc1cc(-c2c(C)cc(C(=O)O)oc2=O)c(C(=O)O)c(O)c1,...,O=C(O)C=Cc1cc(O)c(OC2OC(C(=O)O)C(O)C(O)C2O)cc1,O=C(O)C=Cc1cc(O)c(OC2OC(C(=O)O)C(O)C(O)C2O)cc1,[#8]=[#6](-[#8])-[#6]=[#6]-[#6]1:[#6]:[#6](-[#...,356.074347,LTGOIJNQWZWJRF,C15H16O10,-2.21853,True,False,False
71744,3451,alternarian acid,[M+H]+,1.0,321.06049,NaN,NaN,1.902833,C15H12O8,COc1cc(-c2c(C)cc(C(=O)O)oc2=O)c(C(=O)O)c(O)c1,...,O=C(O)C=Cc1cc(O)c(OC2OC(C(=O)O)C(O)C(O)C2O)cc1,O=C(O)C=Cc1cc(O)c(OC2OC(C(=O)O)C(O)C(O)C2O)cc1,[#8]=[#6](-[#8])-[#6]=[#6]-[#6]1:[#6]:[#6](-[#...,356.074347,LTGOIJNQWZWJRF,C15H16O10,-2.21853,True,False,False
71745,3452,alternarian acid,[M+H]+,1.0,321.06049,NaN,NaN,1.902833,C15H12O8,COc1cc(-c2c(C)cc(C(=O)O)oc2=O)c(C(=O)O)c(O)c1,...,O=C(O)C=Cc1cc(O)c(OC2OC(C(=O)O)C(O)C(O)C2O)cc1,O=C(O)C=Cc1cc(O)c(OC2OC(C(=O)O)C(O)C(O)C2O)cc1,[#8]=[#6](-[#8])-[#6]=[#6]-[#6]1:[#6]:[#6](-[#...,356.074347,LTGOIJNQWZWJRF,C15H16O10,-2.21853,True,False,False


## Tanimoto Similarity

In [123]:
@lru_cache
def fingerprint(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return Chem.GetMorganFingerprintAsBitVect(mol, useChirality=False, radius=2, nBits=2048)

@lru_cache
def tanimoto(fp1, fp2):
    return DataStructs.TanimotoSimilarity(fp1, fp2)

@lru_cache
def tanimoto_smiles(smi1, smi2):
    if smi2 is None:
        return np.nan
    
    fp1 = fingerprint(smi1)
    fp2 = fingerprint(smi2)
    return DataStructs.TanimotoSimilarity(fp1, fp2)

In [124]:
fps1 = [fingerprint(smiles) for smiles in filtered["smiles"]]
fps2 = [fingerprint(smiles) for smiles in filtered["smiles_match"]]
filtered["tanimoto_similarity"] = [tanimoto(fp1, fp2) for fp1, fp2 in zip(fps1, fps2)]
# filtered.sort_values(by="tanimoto_similarity", ascending=False).drop_duplicates(subset="id", keep="first")

In [125]:
filtered

,id,compound_name,adduct,score,precursor_mz,ion_mobility,ccs,rt,mol_formula,smiles,...,isomeric_smiles,smarts,monoisotopic_mass,split_inchikey,formula,logp,match,equal_inchikey,has_identical_inchikey,tanimoto_similarity
0,14441,CHEBI:181155,[M+H]+,1.0,235.16926,NaN,NaN,0.165500,C15H22O2,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,...,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,[#6]-[#6](-[#6])=[#6]1-[#6]-[#6@@]2(-[#6])-[#6...,234.161980,TVDMUSYVWJLIDK,C15H22O2,3.01910,True,False,False,0.186441
1,14441,CHEBI:181155,[M+H]+,1.0,235.16926,NaN,NaN,0.165500,C15H22O2,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,...,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,[#6]-[#6](-[#6])=[#6]1-[#6]-[#6@@]2(-[#6])-[#6...,234.161980,TVDMUSYVWJLIDK,C15H22O2,3.01910,True,False,False,0.186441
2,14441,CHEBI:181155,[M+H]+,1.0,235.16926,NaN,NaN,0.165500,C15H22O2,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,...,CC(C)(O)C1CCC2(C)C3C(O)C4C(C13)C42CO,[#6]-[#6](-[#6])(-[#8])-[#6]1-[#6]-[#6]-[#6]2(...,252.172545,ORUMCUXABQCFEG,C15H24O3,1.01880,True,False,False,0.150000
3,14441,CHEBI:181155,[M+H]+,1.0,235.16926,NaN,NaN,0.165500,C15H22O2,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,...,CC(C)(O)C1CCC2(C)C3C(O)C4C(C13)C42CO,[#6]-[#6](-[#6])(-[#8])-[#6]1-[#6]-[#6]-[#6]2(...,252.172545,ORUMCUXABQCFEG,C15H24O3,1.01880,True,False,False,0.150000
4,14441,CHEBI:181155,[M+H]+,1.0,235.16926,NaN,NaN,0.165500,C15H22O2,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,...,CC1=CC(=O)C(=C(C)C)CC2C1CCC2(C)O,[#6]-[#6]1=[#6]-[#6](=[#8])-[#6](=[#6](-[#6])-...,234.161980,RHBOHEXDGUVIIY,C15H22O2,3.01910,True,False,False,0.186441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71702,9735,Lithospermic acid,[M+NH4]+,1.0,556.14495,NaN,NaN,1.888167,C27H22O12,O=C(/C=C/c1c2c(c(O)cc1)O[C@H](c1cc(O)c(O)cc1)[...,...,O=C(/C=C/c1c2c(c(O)cc1)OC(c1ccc(O)c(O)c1)C2C(=...,[#8]=[#6](/[#6]=[#6]/[#6]1:[#6]2:[#6](:[#6](-[...,538.111126,UJZQBMQZMKFSRV,C27H22O12,-0.22450,True,False,False,1.000000
71743,3451,alternarian acid,[M+H]+,1.0,321.06049,NaN,NaN,1.902833,C15H12O8,COc1cc(-c2c(C)cc(C(=O)O)oc2=O)c(C(=O)O)c(O)c1,...,O=C(O)C=Cc1cc(O)c(OC2OC(C(=O)O)C(O)C(O)C2O)cc1,[#8]=[#6](-[#8])-[#6]=[#6]-[#6]1:[#6]:[#6](-[#...,356.074347,LTGOIJNQWZWJRF,C15H16O10,-2.21853,True,False,False,0.154930
71744,3451,alternarian acid,[M+H]+,1.0,321.06049,NaN,NaN,1.902833,C15H12O8,COc1cc(-c2c(C)cc(C(=O)O)oc2=O)c(C(=O)O)c(O)c1,...,O=C(O)C=Cc1cc(O)c(OC2OC(C(=O)O)C(O)C(O)C2O)cc1,[#8]=[#6](-[#8])-[#6]=[#6]-[#6]1:[#6]:[#6](-[#...,356.074347,LTGOIJNQWZWJRF,C15H16O10,-2.21853,True,False,False,0.154930
71745,3452,alternarian acid,[M+H]+,1.0,321.06049,NaN,NaN,1.902833,C15H12O8,COc1cc(-c2c(C)cc(C(=O)O)oc2=O)c(C(=O)O)c(O)c1,...,O=C(O)C=Cc1cc(O)c(OC2OC(C(=O)O)C(O)C(O)C2O)cc1,[#8]=[#6](-[#8])-[#6]=[#6]-[#6]1:[#6]:[#6](-[#...,356.074347,LTGOIJNQWZWJRF,C15H16O10,-2.21853,True,False,False,0.154930


## myopic Maximum common edge subgraph (MCES)

In [126]:
import my_mces
filtered = my_mces.mces_parallel(filtered, "smiles", "smiles_match", threads=8)

#update df (merged)
# safe df

  0%|          | 0/9539 [00:00<?, ?it/s]

In [127]:
merged = pu.update_dataframes(filtered, merged)
merged

,adduct,adduct_match,canonical_smiles,ccs,ccs_match,compound_name,compound_name_match,equal_inchikey,formula,has_identical_inchikey,...,precursor_mz_match,rt,rt_match,score,score_match,smarts,smiles,smiles_match,split_inchikey,tanimoto_similarity
0,[M+H]+,[M+H]+,CC(C)=C1CC2(C)C(=CC1=O)CCC(O)C2C,NaN,NaN,CHEBI:181155,"NCGC00381379-01_C15H22O2_(4aR,5R,6R)-6-Hydroxy...",False,C15H22O2,False,...,235.16900,0.1655,NaN,1.0,0.823,[#6]-[#6](-[#6])=[#6]1-[#6]-[#6@@]2(-[#6])-[#6...,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,TVDMUSYVWJLIDK,0.186441
1,[M+H]+,[M+H]+,CC(C)=C1CC2(C)C(=CC1=O)CCC(O)C2C,NaN,NaN,CHEBI:181155,"NCGC00381379-01_C15H22O2_(4aR,5R,6R)-6-Hydroxy...",False,C15H22O2,False,...,235.16900,0.1655,NaN,1.0,0.823,[#6]-[#6](-[#6])=[#6]1-[#6]-[#6@@]2(-[#6])-[#6...,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,TVDMUSYVWJLIDK,0.186441
2,[M+H]+,[M-H2O+H]+,CC(C)(O)C1CCC2(C)C3C(O)C4C(C13)C42CO,NaN,NaN,CHEBI:181155,"NCGC00385861-01_C15H24O3_1,2,4-Metheno-1H-inde...",False,C15H24O3,False,...,235.16900,0.1655,NaN,1.0,0.726,[#6]-[#6](-[#6])(-[#8])-[#6]1-[#6]-[#6]-[#6]2(...,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,CC(C)(O)C1CCC2(C)C3C(O)C4C(C13)C42CO,ORUMCUXABQCFEG,0.150000
3,[M+H]+,[M-H2O+H]+,CC(C)(O)C1CCC2(C)C3C(O)C4C(C13)C42CO,NaN,NaN,CHEBI:181155,"NCGC00385861-01_C15H24O3_1,2,4-Metheno-1H-inde...",False,C15H24O3,False,...,235.16900,0.1655,NaN,1.0,0.726,[#6]-[#6](-[#6])(-[#8])-[#6]1-[#6]-[#6]-[#6]2(...,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,CC(C)(O)C1CCC2(C)C3C(O)C4C(C13)C42CO,ORUMCUXABQCFEG,0.150000
4,[M+H]+,[M+H]+,CC1=CC(=O)C(=C(C)C)CC2C1CCC2(C)O,NaN,NaN,CHEBI:181155,NCGC00385860-01_C15H22O2_3-Hydroxy-5-isopropyl...,False,C15H22O2,False,...,235.16900,0.1655,NaN,1.0,0.722,[#6]-[#6]1=[#6]-[#6](=[#8])-[#6](=[#6](-[#6])-...,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,CC1=CC(=O)C(=C(C)C)CC2C1CCC2(C)O,RHBOHEXDGUVIIY,0.186441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71990,[M+H]+,[M+H]+,CC(C)=C1CC2(C)C(=CC1=O)CCC(O)C2C,NaN,NaN,ISOPETASOL,"NCGC00381379-01_C15H22O2_(4aR,5R,6R)-6-Hydroxy...",True,C15H22O2,True,...,235.16900,2.4680,NaN,1.0,0.674,[#6]-[#6](-[#6])=[#6]1-[#6]-[#6@@]2(-[#6])-[#6...,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,TVDMUSYVWJLIDK,NaN
71991,[M+H]+,[M+H]+,CC1=CC2(O)OC3(CC2=C(C)C)C(C)CCC13,NaN,NaN,ISOPETASOL,Curcumenol,False,C15H22O2,True,...,235.16900,2.4680,NaN,1.0,0.646,[#6]-[#6]1=[#6]-[#6]2(-[#8])-[#8]-[#6]3(-[#6]-...,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,CC1=CC2(O)OC3(CC2=C(C)C)C(C)CCC13,ISFMXVMWEWLJGJ,NaN
71992,[M+H]+,[M+H]+,CC1=CC2(O)OC3(CC2=C(C)C)C(C)CCC13,NaN,NaN,ISOPETASOL,Curcumenol,False,C15H22O2,True,...,235.16925,2.4680,NaN,1.0,0.646,[#6]-[#6]1=[#6]-[#6]2(-[#8])-[#8]-[#6]3(-[#6]-...,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,CC1=CC2(O)OC3(CC2=C(C)C)C(C)CCC13,ISFMXVMWEWLJGJ,NaN
71993,[M+H]+,[M-H2O+H]+,C=C1CC(O)C(O)C(C)C(=O)CC2C1CC2(C)C,NaN,NaN,ISOPETASOL,"Massbank:AC000285 Walleminone|(1R,4S,5S,6R,9R)...",False,C15H24O3,True,...,235.16900,2.4680,NaN,1.0,0.634,[#6]=[#6]1-[#6]-[#6@@H](-[#8])-[#6@@H](-[#8])-...,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,C=C1C[C@@H](O)[C@@H](O)[C@H](C)C(=O)C[C@@H]2[C...,NGQXJSTYWWTPOG,NaN


In [ ]:
merged["tanimoto_similarity"] = np.where(merged['equal_inchikey'], 1, merged['tanimoto_similarity'])
# merged["tanimoto_similarity"] = np.where(merged['match'] != True, -1, merged['tanimoto_similarity'])
merged["mces"] = np.where(merged['equal_inchikey'], 0, merged['mces'])
# merged["mces"] = np.where(merged['match']  != True, 1000, merged['mces'])

In [10]:
pu.save_dataframe(merged, outfile)

NameError: name 'merged' is not defined

In [54]:
file = r"C:\git\msn_library\data\library_matching\20231031\match_top10_20231031_nih_library_pos_all_lib_MS2.mgf__19241_spectra__single_scans_comparison.csv"

In [55]:
df = pu.read_dataframe(file)

In [56]:
df["tanimoto_similarity"] = np.where(df['equal_inchikey'], 1, df['tanimoto_similarity'])
df["mces"] = np.where(df['equal_inchikey'], 0, df['mces'])

In [57]:
df

,adduct,adduct_match,canonical_smiles,ccs,ccs_match,compound_name,compound_name_match,equal_inchikey,formula,has_identical_inchikey,...,precursor_mz_match,rt,rt_match,score,score_match,smarts,smiles,smiles_match,split_inchikey,tanimoto_similarity
0,[M+H]+,[M+H]+,CC(C)=C1CC2(C)C(=CC1=O)CCC(O)C2C,NaN,NaN,CHEBI:181155,"NCGC00381379-01_C15H22O2_(4aR,5R,6R)-6-Hydroxy...",False,C15H22O2,False,...,235.16900,0.1655,NaN,1.0,0.823,[#6]-[#6](-[#6])=[#6]1-[#6]-[#6@@]2(-[#6])-[#6...,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,TVDMUSYVWJLIDK,0.186441
1,[M+H]+,[M+H]+,CC(C)=C1CC2(C)C(=CC1=O)CCC(O)C2C,NaN,NaN,CHEBI:181155,"NCGC00381379-01_C15H22O2_(4aR,5R,6R)-6-Hydroxy...",False,C15H22O2,False,...,235.16900,0.1655,NaN,1.0,0.823,[#6]-[#6](-[#6])=[#6]1-[#6]-[#6@@]2(-[#6])-[#6...,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,TVDMUSYVWJLIDK,0.186441
2,[M+H]+,[M-H2O+H]+,CC(C)(O)C1CCC2(C)C3C(O)C4C(C13)C42CO,NaN,NaN,CHEBI:181155,"NCGC00385861-01_C15H24O3_1,2,4-Metheno-1H-inde...",False,C15H24O3,False,...,235.16900,0.1655,NaN,1.0,0.726,[#6]-[#6](-[#6])(-[#8])-[#6]1-[#6]-[#6]-[#6]2(...,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,CC(C)(O)C1CCC2(C)C3C(O)C4C(C13)C42CO,ORUMCUXABQCFEG,0.150000
3,[M+H]+,[M-H2O+H]+,CC(C)(O)C1CCC2(C)C3C(O)C4C(C13)C42CO,NaN,NaN,CHEBI:181155,"NCGC00385861-01_C15H24O3_1,2,4-Metheno-1H-inde...",False,C15H24O3,False,...,235.16900,0.1655,NaN,1.0,0.726,[#6]-[#6](-[#6])(-[#8])-[#6]1-[#6]-[#6]-[#6]2(...,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,CC(C)(O)C1CCC2(C)C3C(O)C4C(C13)C42CO,ORUMCUXABQCFEG,0.150000
4,[M+H]+,[M+H]+,CC1=CC(=O)C(=C(C)C)CC2C1CCC2(C)O,NaN,NaN,CHEBI:181155,NCGC00385860-01_C15H22O2_3-Hydroxy-5-isopropyl...,False,C15H22O2,False,...,235.16900,0.1655,NaN,1.0,0.722,[#6]-[#6]1=[#6]-[#6](=[#8])-[#6](=[#6](-[#6])-...,CC1CCC2C(C(=O)O)=CC3CC(C)(C)CC132,CC1=CC(=O)C(=C(C)C)CC2C1CCC2(C)O,RHBOHEXDGUVIIY,0.186441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71990,[M+H]+,[M+H]+,CC(C)=C1CC2(C)C(=CC1=O)CCC(O)C2C,NaN,NaN,ISOPETASOL,"NCGC00381379-01_C15H22O2_(4aR,5R,6R)-6-Hydroxy...",True,C15H22O2,True,...,235.16900,2.4680,NaN,1.0,0.674,[#6]-[#6](-[#6])=[#6]1-[#6]-[#6@@]2(-[#6])-[#6...,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,TVDMUSYVWJLIDK,1.000000
71991,[M+H]+,[M+H]+,CC1=CC2(O)OC3(CC2=C(C)C)C(C)CCC13,NaN,NaN,ISOPETASOL,Curcumenol,False,C15H22O2,True,...,235.16900,2.4680,NaN,1.0,0.646,[#6]-[#6]1=[#6]-[#6]2(-[#8])-[#8]-[#6]3(-[#6]-...,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,CC1=CC2(O)OC3(CC2=C(C)C)C(C)CCC13,ISFMXVMWEWLJGJ,NaN
71992,[M+H]+,[M+H]+,CC1=CC2(O)OC3(CC2=C(C)C)C(C)CCC13,NaN,NaN,ISOPETASOL,Curcumenol,False,C15H22O2,True,...,235.16925,2.4680,NaN,1.0,0.646,[#6]-[#6]1=[#6]-[#6]2(-[#8])-[#8]-[#6]3(-[#6]-...,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,CC1=CC2(O)OC3(CC2=C(C)C)C(C)CCC13,ISFMXVMWEWLJGJ,NaN
71993,[M+H]+,[M-H2O+H]+,C=C1CC(O)C(O)C(C)C(=O)CC2C1CC2(C)C,NaN,NaN,ISOPETASOL,"Massbank:AC000285 Walleminone|(1R,4S,5S,6R,9R)...",False,C15H24O3,True,...,235.16900,2.4680,NaN,1.0,0.634,[#6]=[#6]1-[#6]-[#6@@H](-[#8])-[#6@@H](-[#8])-...,CC(C)=C1C[C@@]2(C)C(=CC1=O)CC[C@@H](O)[C@@H]2C,C=C1C[C@@H](O)[C@@H](O)[C@H](C)C(=O)C[C@@H]2[C...,NGQXJSTYWWTPOG,NaN


In [58]:
pu.save_dataframe(df, file)

In [129]:
stats = {
    "MS2 scans": len(original_df),
    "unique InChIKeys": original_df["inchikey"].nunique(),
    "Any spectral match": len(match_df.groupby("id")),
    "Matching InChIKey": len(id_with_identity_match),
    "Tanimoto 1": len(filtered[filtered["tanimoto_similarity"] ==1].drop_duplicates(subset="id", keep="first"))+len(id_with_identity_match),
    "unmatched scans": len(merged[merged["match"] != True].drop_duplicates(subset="id")),
    "unmatched InChIKeys": len(merged[merged["match"] != True].drop_duplicates(subset="inchikey"))
}


stats["dadsad"] = "dawdsadwa"

pd.DataFrame([stats])

,MS2 scans,unique InChIKeys,Any spectral match,Matching InChIKey,Tanimoto 1,unmatched scans,unmatched InChIKeys,dadsad
0,19241,2305,12509,10725,11488,6732,1577,dawdsadwa


In [130]:
for v in stats.values():
    print(v)

19241
2305
12509
10725
11488
6732
1577
dawdsadwa
